This file is part of the [test suite](./tests) and will be moved there when [nbval#116](https://github.com/computationalmodelling/nbval/issues/116#issuecomment-793148404) is fixed.

See [DEMO.ipynb](DEMO.ipynb) instead for notebook examples.

In [1]:
from functools import partial
from time import sleep

from tqdm.notebook import tqdm_notebook
from tqdm.notebook import tnrange

# avoid displaying widgets by default (pollutes output cells)
tqdm = partial(tqdm_notebook, display=False)
trange = partial(tnrange, display=False)

In [2]:
help(tqdm_notebook.display)

Help on function display in module tqdm.notebook:

display(self, msg=None, pos=None, close=False, bar_style=None, check_delay=True)
    Use `self.sp` to display `msg` in the specified `pos`.
    
    Consider overloading this function when inheriting to use e.g.:
    `self.some_frontend(**self.format_dict)` instead of `self.sp`.
    
    Parameters
    ----------
    msg  : str, optional. What to display (default: `repr(self)`).
    pos  : int, optional. Position to `moveto`
      (default: `abs(self.pos)`).



In [3]:
# NBVAL_TEST_NAME: basic use
with tqdm_notebook(range(9)) as t:
    for i in t:
        print(i)
assert t.container.children[1].bar_style == 'success'

t = tqdm_notebook(total=9)
t.update()
t.refresh()

  0%|          | 0/9 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8


  0%|          | 0/9 [00:00<?, ?it/s]

True

In [4]:
# NBVAL_TEST_NAME: reset
print(t)
t.reset(total=5)
t.update(1)
print(t)

 20%|██        | 1/5 [00:00<00:00, 341.50it/s]


In [5]:
# NBVAL_TEST_NAME: bar_style
assert t.container.children[1].bar_style != 'danger'
t.close()
assert t.container.children[1].bar_style == 'danger'

In [6]:
# NBVAL_TEST_NAME: repr
with trange(1, 9) as t:
    print(t)
    print(t.container)
    it = iter(t)
    print(next(it))
    print(t)
    print(t.container)

  0%|          | 0/8 [00:00<?, ?it/s]
1
  0%|          | 0/8 [00:00<?, ?it/s]


In [7]:
t = trange(9)

In [8]:
# NBVAL_TEST_NAME: display pre
print(t)
print(t.container)

  0%|          | 0/9 [00:00<?, ?it/s]


In [9]:
for i in t:
    pass

In [10]:
# NBVAL_TEST_NAME: display post
print(t)
print(t.container)

100%|##########| 9/9 [00:00<00:00, 131.02it/s]


In [11]:
# NBVAL_TEST_NAME: no total
with tqdm(desc="no total") as t:
    print(t)
    t.update()
    print(t)

no total: 0it [00:00, ?it/s]
no total: 1it [00:00, 47.83it/s]


In [12]:
# NBVAL_TEST_NAME: ncols
with trange(9, ncols=66) as t:
    print(t)
    for i in t:
        if i == 1:
            break
    print(t)

 11%|███▍                           | 1/9 [00:00<00:00, 45.06it/s]


In [13]:
# NBVAL_TEST_NAME: leave
def is_hidden(widget):
    return ('hidden', False, None) == (
        getattr(getattr(widget, "layout", None), "visibility", 'visible'), # ipyw>=8
        getattr(widget, "visible", False), getattr(widget, "_ipython_display_", None)) # ipyw<8

assert not is_hidden(t.container)
for total in (1, 9):
    with tqdm(total=total, leave=False) as t:
        print(t)
        t.update()
        print(t)
    assert total != 1 or is_hidden(t.container)

 11%|█         | 1/9 [00:00<00:00, 57.43it/s]


In [14]:
# NBVAL_TEST_NAME: no total
with tqdm() as t:
    print(t)
    t.update()
    print(t)

0it [00:00, ?it/s]
1it [00:00, 47.87it/s]


In [15]:
# NBVAL_TEST_NAME: reset and disable
for disable in (None, True):
    print("disable:", disable)
    with tqdm(total=1, disable=disable) as t:
        print(t)
        t.update()
        print(t)

        t.reset(total=9)
        print(t)
        t.update()
        print(t)
    with tqdm(disable=disable) as t:
        print(t)
        t.update()
        print(t)

        t.reset(total=1)
        print(t)
        t.update()
        print(t)

disable: None
 11%|█         | 1/9 [00:00<00:00, 1752.01it/s]
0it [00:00, ?it/s]
1it [00:00, 35.88it/s]
100%|██████████| 1/1 [00:00<00:00, 1880.85it/s]
disable: True
  0%|          | 0/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]


In [16]:
# NBVAL_TEST_NAME: bar_format
with tqdm(total=1, bar_format='{l_bar}{r_bar}') as t:
    print(t)
    t.update()
    print(t)

with tqdm(total=1, bar_format='{l_bar}{bar}') as t:
    print(t)
    t.update()
    print(t)

100%|| 1/1 [00:00<00:00, 32.57it/s]
  0%|          
100%|██████████


In [17]:
# NBVAL_TEST_NAME: colour
assert t.colour != 'yellow'
with tqdm(total=1, colour='yellow') as t:
    print(t)
    t.update()
    print(t)
    assert t.colour == 'yellow'

100%|██████████| 1/1 [00:00<00:00, 47.14it/s]


In [18]:
# NBVAL_TEST_NAME: delay no trigger
with tqdm_notebook(total=1, delay=10) as t:
    t.update()

In [19]:
# NBVAL_TEST_NAME: delay trigger
with tqdm_notebook(total=1, delay=0.1) as t:
    sleep(0.1)
    t.update()

100%|##########| 1/1 [00:00<00:00,  7.68it/s]